In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from datetime import datetime, timedelta
import os
import pandas as pd
from bs4 import BeautifulSoup

start_time = time.time()

snapshot_url = "https://www.mse.mk/mk/stats/symbolhistory/KMB"
original_url = "https://www.mse.mk/mk/stats/symbolhistory/"

browser = webdriver.Chrome()
browser.get(snapshot_url)

list_codes = browser.find_element(By.ID, "Code").find_elements(By.TAG_NAME, "option")
codes = []
for c in list_codes:
    c = c.text.strip()
    if not any(char.isdigit() for char in c):
        codes.append(c)
print(len(codes))

data_mse = "mse_data.csv"
file_path = "mse_last_access_dates.csv"

if not os.path.exists(file_path):
    mse_last_access = pd.DataFrame(columns=['Code', 'LastAccess'])
    mse_last_access.to_csv(file_path, index=False)
else:
    mse_last_access = pd.read_csv(file_path)

if not os.path.exists(data_mse):
    columns = ['Code', 'Date', 'PriceLastTransaction', 'Max', 'Min', 'AveragePrice', 
               'GainPercent', 'Volume', 'GainBestDenari', 'TotalGain']
    mse_data_file = pd.DataFrame(columns=columns)
    mse_data_file.to_csv(data_mse, index=False)
else:
    mse_data_file = pd.read_csv(data_mse)

data_dictionary = []

def fromToDateData(date_range, code, retry_count=1):
    WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.ID, "FromDate")))
    fromDate, toDate = date_range.split(' - ')
    
    try:
        toDateSelector = WebDriverWait(browser, 3).until(EC.visibility_of_element_located((By.ID, "ToDate")))
        fromDateSelector = WebDriverWait(browser, 3).until(EC.visibility_of_element_located((By.ID, "FromDate")))
        
        toDateSelector.clear()
        toDateSelector.send_keys(toDate)
        fromDateSelector.clear()
        fromDateSelector.send_keys(fromDate)
        
        fromDateSelector.send_keys(Keys.ENTER)
        fromDateSelector.send_keys(Keys.ENTER)
        
        try:
            WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.ID, "resultsTable")))
        except TimeoutException:
            print("Table did not load. Skipping date range.")
            return False 
        
        page_source = browser.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        table = soup.select_one("#resultsTable")
        
        if table:
            rows = table.select("tr")
            for row in rows[1:]:
                cells = row.select("td")
                
                if any(not cell.text.strip() for cell in cells):
                    continue
                try:
                    parsed_date = datetime.strptime(cells[0].strip(), "%d.%m.%Y")  
                    formatted_date = parsed_date.strftime("%d.%m.%Y") 
                    
                    data = {
                        "Code": code,
                        "Date": formatted_date,
                        "PriceLastTransaction": float(cells[1].text.strip().replace(".", "").replace(",", ".")),
                        "Max": float(cells[2].text.strip().replace(".", "").replace(",", ".")),
                        "Min": float(cells[3].text.strip().replace(".", "").replace(",", ".")),
                        "AveragePrice": float(cells[4].text.strip().replace(".", "").replace(",", ".")),
                        "GainPercent": float(cells[5].text.strip().replace(".", "").replace(",", ".")),
                        "Volume": int(cells[6].text.strip().replace(".", "").replace(",", "")),
                        "GainBestDenari": float(cells[7].text.strip().replace(".", "").replace(",", ".")),
                        "TotalGain": float(cells[8].text.strip().replace(".", "").replace(",", "."))
                    }
                    
                    data_dictionary.append(data)
                except ValueError:
                    print("Invalid data conversion. Skipping row.")
                    continue

            return True

        else:
            print("No table found. Skipping date range.")
            return False

    except StaleElementReferenceException:
        if retry_count > 0:
            print(f"Stale element detected. Retrying... ({retry_count} retries left)")
            time.sleep(1)
            return fromToDateData(date_range, code, retry_count - 1)
        else:
            print("Max retries reached. Skipping date range.")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

def generate_date_ranges(days_back, years):
    date_ranges = []
    end_date = datetime.today()
    for _ in range(years * (365 // days_back)):
        start_date = end_date - timedelta(days=days_back)
        date_ranges.append(f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}")
        end_date = start_date - timedelta(days=1)
    return date_ranges

date_ranges = generate_date_ranges(365, 10)

for c in codes:
    get_url = original_url + f"{c}"
    browser.get(get_url)

    today_date = datetime.today().strftime('%d.%m.%Y')

    if c in mse_data_file['Code'].values:
        if not mse_last_access[mse_last_access['Code'] == c].empty:
            last_access = mse_last_access[mse_last_access['Code'] == c].iloc[0]['LastAccess']
        
            if last_access == today_date:
                print(f"Skipping {c} as it was already accessed today.")
            else:
                date = f"{last_access} - {today_date}"
                fromToDateData(date, c)

                mse_last_access.loc[mse_last_access['Code'] == c, 'LastAccess'] = today_date
    else:
        for date in date_ranges:
            result = fromToDateData(date, c)
            if result is False:
                print(f"No table found for {c}, skipping.")
                break

        mse_last_access = pd.concat([mse_last_access, pd.DataFrame({"Code": [c], "LastAccess": [today_date]})], ignore_index=True)

mse_last_access.to_csv(file_path, index=False)

if data_dictionary:
    new_data_df = pd.DataFrame(data_dictionary)
    
    if os.path.exists(data_mse):
        existing_data_df = pd.read_csv(data_mse)
    else:
        columns = ['Code', 'Date', 'PriceLastTransaction', 'Max', 'Min', 'AveragePrice', 
                   'GainPercent', 'Volume', 'GainBestDenari', 'TotalGain']
        existing_data_df = pd.DataFrame(columns=columns)

    existing_data_df['Date'] = pd.to_datetime(existing_data_df['Date'], errors='coerce')
    new_data_df['Date'] = pd.to_datetime(new_data_df['Date'], errors='coerce')

    merged_data_df = pd.concat([existing_data_df, new_data_df]).drop_duplicates(subset=['Code', 'Date'])
    merged_data_df = merged_data_df.sort_values(by=['Code', 'Date'])
    merged_data_df.to_csv(data_mse, index=False)

    print("Data successfully merged and saved to mse_data_retry.csv.")
else:
    print("No new data to add, skipping merge.")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")